In [1]:
import os
import pandas as pd
import numpy as np 
#get rid of max rows limit
pd.set_option('display.max_rows', None)

#list all files in the directory
path_to_files = './patent_data_files/'
files = os.listdir(path_to_files)
files

['.DS_Store',
 'CN109517041_extracted.csv',
 'WO2012004664_extracted.csv',
 'WO2012125973_extracted.csv',
 'WO2013173706_extracted.csv',
 'WO2014165277_extracted.csv',
 'WO2016140859_extracted.csv',
 'WO2017219081_extracted.csv']

In [2]:
# import all the csv and xlsx files into separate dataframes
df_list = []
for file in files:
    if file.endswith('.csv'):
        print(file)
        df_list.append(pd.read_csv(path_to_files+file))

CN109517041_extracted.csv
WO2012004664_extracted.csv
WO2012125973_extracted.csv
WO2013173706_extracted.csv
WO2014165277_extracted.csv
WO2016140859_extracted.csv
WO2017219081_extracted.csv


## Preprocessing

In [3]:
#convert ['Nav1.7 IC50 (nM)'] column to float in all the dataframes
#if Nav1.7 IC50 (nM) contains a > sign, remove it 
for df in df_list:
    #if type string and contains a > sign, remove it
    for x in ['Nav1.7 IC50 (nM)', 'Nav1.4 IC50 (nM)', 'Nav1.2 IC50 (nM)']:
        #handle key error
        if x not in df.columns:
            continue
        
        if df[x].dtype == 'O':
            df[x] = df[x].str.replace('>', '')
        df[x] = df[x].astype(float)

    #print number of duplicate rows
    print('Number of duplicate rows: ', df.duplicated().sum())
    #drop duplicate rows
    df = df.drop_duplicates(inplace=True)

Number of duplicate rows:  0
Number of duplicate rows:  0
Number of duplicate rows:  0
Number of duplicate rows:  44
Number of duplicate rows:  0
Number of duplicate rows:  0
Number of duplicate rows:  0


In [4]:
# combine all the dataframes, keeping only the columns of interest
columns_list= ["Sequence ID", "Sequence", "Assay", "Reference", "Nav1.7 IC50 (nM)", "Nav1.2 IC50 (nM)", "Nav1.4 IC50 (nM)", "MOD_RES_POSITIONS", "MOD_RES_NOTES", "REGION_NOTES"] 

df = pd.concat(df_list, ignore_index=True)
df = df[columns_list]

df.head()
    

,Sequence ID,Sequence,Assay,Reference,Nav1.7 IC50 (nM),Nav1.2 IC50 (nM),Nav1.4 IC50 (nM),MOD_RES_POSITIONS,MOD_RES_NOTES,REGION_NOTES
0,SEQ ID NO: 1 (GpTx-1),DCLGFMRKCIPDNDKCCRPNLVCSRTHKWCKYVF,Unknown,CN109517041,90.0,NaN,3.7,NaN,NaN,NaN
1,SEQ ID NO: 3,DCLGAFRKCIPDNDKCCRPNLVCSRLHRWCKYVF,Unknown,CN109517041,1.6,NaN,1900.0,NaN,NaN,NaN
2,SEQ ID NO: 4,DCLGFMRKCEPDNDKCCRPNLVCSRTHKWCKYVF,Unknown,CN109517041,2.1,NaN,1300.0,NaN,NaN,NaN
3,SEQ ID NO: 5,DCLGFMRKCIEDNDKCCRPNLVCSRTHKWCKYVF,Unknown,CN109517041,2.5,NaN,1400.0,NaN,NaN,NaN
4,SEQ ID NO: 6,DCLGFMRKCIPDNDKCCKPNLVCSRTHKWCKYVF,Unknown,CN109517041,1.6,NaN,5100.0,NaN,NaN,NaN


### Some Cleaning

In [5]:
print(df['Assay'].unique())

#where assay == 'FLIPR Tetra ', replace it with 'FLIPR Tetra'
df['Assay'] = df['Assay'].replace('FLIPR Tetra ', 'FLIPR Tetra')

#where assay == 'lonWorks Quattro' replace it with 'IonWorks Quattro'
df['Assay'] = df['Assay'].replace('lonWorks Quattro', 'IonWorks Quattro')

#rename Nav1.7 IC50 (nM) to IC50
df = df.rename(columns={'Nav1.7 IC50 (nM)': 'IC50'})

#only keep rows where Nav1.7 IC50 (nM) is not null
df = df[df['IC50'].notnull()]

#print the unique values of Assay
print(df['Assay'].unique())

['Unknown' 'FLIPR Tetra' 'lonWorks Quattro' 'PatchXpress' 'Qpatch'
 'PatchXpress Tonic IC50' 'FLIPR Tetra ']
['Unknown' 'FLIPR Tetra' 'IonWorks Quattro' 'PatchXpress' 'Qpatch'
 'PatchXpress Tonic IC50']


### Add a new column: Variants_of: 

In [6]:
# Mapping of Reference values to Variants_of values
variants_of_mapping = {
    'WO2012004664': 'Protoxin II',
    'WO2017219081': 'Pnc1a',
    'CN109517041': 'GpTx-1',
    'WO2013173706': 'Huwentoxin-IV',
    'WO2012125973': 'GpTx-1',
    'WO2014165277': 'JzTx-V',
    'WO2016140859': 'Protoxin II'
}

# Add the Variants_of column based on the mapping
df['Variants_of'] = df['Reference'].map(variants_of_mapping)

In [7]:
df.describe()

,IC50,Nav1.2 IC50 (nM),Nav1.4 IC50 (nM)
count,2149.000000,340.000000,1366.000000
mean,848.505972,1580.667647,7285.407028
std,2281.580822,5872.165150,11463.844999
min,0.021000,8.000000,0.000000
25%,18.000000,90.000000,700.000000
50%,80.000000,239.800000,4255.000000
75%,310.000000,790.000000,7000.000000
max,23610.000000,73939.000000,50000.000000


### Get rid of special N-term or C-term modifications

In [8]:
print(df['REGION_NOTES'].value_counts(dropna=False))

#get rid of everything except ['C-term Amide'] and NaN
#df = df[df['REGION_NOTES'].isin(["['C-term Amide']", np.nan])]

#print(df['REGION_NOTES'].value_counts(dropna=False))


REGION_NOTES
['C-term Amide']                                                              1372
NaN                                                                            681
['C-term Free Acid']                                                            72
['N-term 4-Pen', 'C-term Amide']                                                 8
['N-term Acetyl', 'C-term Amide']                                                4
['N-term ((2-Hydroxyethyl)thio)acetamide-NPEG11-triazole', 'C-term Amide']       2
['C-term NH-butyl']                                                              2
['C-term NH-methyl']                                                             2
['N-term Biotin', 'C-term Amide']                                                1
['N-term DOTA', 'C-term Amide']                                                  1
['C-term FreeAcid']                                                              1
['N-term Acetyl']                                                         

In [9]:
print(df['MOD_RES_NOTES'].value_counts(dropna=False))



MOD_RES_NOTES
NaN                                                                     1318
['Pra', 'Nle']                                                            83
['Atz(amino-PEG10)']                                                      79
['1-Nal']                                                                 78
['Nle']                                                                   33
['CyA', 'Nle', 'Pra']                                                     17
['Nva']                                                                   17
['2PAL']                                                                  16
['Pra']                                                                   16
['Atz(amino-PEG10)', '1-Nal']                                             16
['Cit']                                                                   16
['2-Abu']                                                                 15
['4CO2-Phe']                                                  

In [10]:
import ast
#convert string to list
df['MOD_RES_NOTES'] = df['MOD_RES_NOTES'].apply(lambda x: ast.literal_eval(x) if type(x) == str else x)



In [11]:
df.head()

,Sequence ID,Sequence,Assay,Reference,IC50,Nav1.2 IC50 (nM),Nav1.4 IC50 (nM),MOD_RES_POSITIONS,MOD_RES_NOTES,REGION_NOTES,Variants_of
0,SEQ ID NO: 1 (GpTx-1),DCLGFMRKCIPDNDKCCRPNLVCSRTHKWCKYVF,Unknown,CN109517041,90.0,NaN,3.7,NaN,NaN,NaN,GpTx-1
1,SEQ ID NO: 3,DCLGAFRKCIPDNDKCCRPNLVCSRLHRWCKYVF,Unknown,CN109517041,1.6,NaN,1900.0,NaN,NaN,NaN,GpTx-1
2,SEQ ID NO: 4,DCLGFMRKCEPDNDKCCRPNLVCSRTHKWCKYVF,Unknown,CN109517041,2.1,NaN,1300.0,NaN,NaN,NaN,GpTx-1
3,SEQ ID NO: 5,DCLGFMRKCIEDNDKCCRPNLVCSRTHKWCKYVF,Unknown,CN109517041,2.5,NaN,1400.0,NaN,NaN,NaN,GpTx-1
4,SEQ ID NO: 6,DCLGFMRKCIPDNDKCCKPNLVCSRTHKWCKYVF,Unknown,CN109517041,1.6,NaN,5100.0,NaN,NaN,NaN,GpTx-1


In [12]:
protoxin = df[df['Reference'] == 'WO2016140859']
#print all with list length > 1
protoxin.groupby('Sequence')['Sequence ID'].apply(set).loc[lambda x: x.str.len() > 1]


Sequence
GPQCQKWMQTCDAERKCCEGFSCTLWCKKKLW                  {129, 130}
GPQCQKWMQTCDAERKCCEGFVCRLWCKKKLW                   {56, 111}
GPQCQKWMQTCDRERKCCEGFVCTLWCRKKLW    {78, 116, 117, 118, 119}
GPQCQKWMQTCDRTRKCCEGFVCTLWCRKKLW                  {122, 123}
GPSCQKWFWTCDAERKCCEGLVCRLWCKKKLW                   {112, 66}
GPYCQKWMQTCDANRKCCEGFSCRLWCKKKLW                  {324, 325}
QCQKWMQTCDRERKCCEGFVCTLWCRKKLW                    {120, 121}
SCQKWMQTCDAERKCCEGFVCRLWCKKKLW                    {109, 110}
Name: Sequence ID, dtype: object

## unmodified residue sequences

In [13]:
#make a dataframe of sequences where MOD_RES_POSITIONS is null
df_no_mod_res = df[df['MOD_RES_POSITIONS'].isnull()].reset_index(drop=True)

#drop the MOD_RES_POSITIONS and MOD_RES_NOTES columns
df_no_mod_res = df_no_mod_res.drop(columns=['MOD_RES_POSITIONS', 'MOD_RES_NOTES'])

#print the unique values of Reference
print(df_no_mod_res.head())
print(len(df_no_mod_res))

#write to csv
df_no_mod_res.to_csv('../Data/raw/df_no_mod_res.csv', index=False)



             Sequence ID                            Sequence    Assay  \
0  SEQ ID NO: 1 (GpTx-1)  DCLGFMRKCIPDNDKCCRPNLVCSRTHKWCKYVF  Unknown   
1           SEQ ID NO: 3  DCLGAFRKCIPDNDKCCRPNLVCSRLHRWCKYVF  Unknown   
2           SEQ ID NO: 4  DCLGFMRKCEPDNDKCCRPNLVCSRTHKWCKYVF  Unknown   
3           SEQ ID NO: 5  DCLGFMRKCIEDNDKCCRPNLVCSRTHKWCKYVF  Unknown   
4           SEQ ID NO: 6  DCLGFMRKCIPDNDKCCKPNLVCSRTHKWCKYVF  Unknown   

     Reference  IC50  Nav1.2 IC50 (nM)  Nav1.4 IC50 (nM) REGION_NOTES  \
0  CN109517041  90.0               NaN               3.7          NaN   
1  CN109517041   1.6               NaN            1900.0          NaN   
2  CN109517041   2.1               NaN            1300.0          NaN   
3  CN109517041   2.5               NaN            1400.0          NaN   
4  CN109517041   1.6               NaN            5100.0          NaN   

  Variants_of  
0      GpTx-1  
1      GpTx-1  
2      GpTx-1  
3      GpTx-1  
4      GpTx-1  
1318


## Whole Dataset

In [14]:
#df.to_csv('df_complete.csv', index=False)